In [225]:
import numpy as np
import scipy as sp
import math
import matplotlib.pyplot as plt

np.random.seed(123)

%run ./DiffusionLunarKF.ipynb
%run ./CentralizedLunarKF.ipynb
%run ./FilterComparison.ipynb
%run ./VectorFormatting.ipynb

[0.2808067537688888, 0.11536742677858898, 0.09146337207788346, 0.22228249836798888, 0.30138564230254933, 0.5874526275750482, 0.5332787972708654, 0.6243884898665294]
[[0.29007995 0.28080675 0.11536743 0.09146337 0.2222825 ]
 [0.69861436 0.30138564 0.         0.         0.        ]
 [0.41254737 0.         0.58745263 0.         0.        ]
 [0.4667212  0.         0.         0.5332788  0.        ]
 [0.37561151 0.         0.         0.         0.62438849]]
Row Sums:  [1.0, 1.0, 1.0, 1.0, 1.0]


In [263]:
def MSE(vec):
    #C just is a reference for where we know there are edges in our network
    avg = 0
    for i in range(15):
        adj = np.array([[1,1,1,1,1],[1,1,0,0,0],[1,0,1,0,0],[1,0,0,1,0],[1,0,0,0,1]])
        # adj = np.array([[1,0,1],[0,1,0],[1,0,1]])

        C = convert_to_adj(vec,adj)

        n = len(C)

        D = 3*adj
        D_ckf = 3*np.ones((n,n))
        np.random.seed(i)
        true_biases = np.array([[np.random.normal(0,np.sqrt(12/(c**2))) for _ in range(n)]]).T
        true_drifts = np.array([[np.random.normal(0,np.sqrt(0.1/(c**2))) for _ in range(n)]]).T
        # true_drifts = np.array([[0 for _ in range(n)]]).T

        F = np.array([[1,dt],[0,1]])
        F_full = np.kron(np.eye(n),F)

        x = c*np.vstack(tuple([np.array([true_biases[i],true_drifts[i]]) for i in range(n)]))

        # random initial estimates for each node

        x0 = [np.array([[np.random.normal(0,np.sqrt(12))],[np.random.normal(0,np.sqrt(0.1))]]) for i in range(n)]
        x0_cf = np.vstack(tuple(x0))
        # x0 = [np.array([[0],[0]]) for _ in range(n)]

        P = [100*np.copy(R(1)) for _ in range(n)]
        P_prev = np.block([[P[i] if i==j else np.zeros((2,2)) for j in range(n)] for i in range(n)])

        stations = [Station(i) for i in range(n)]

        filter_initialize(stations,D,x0,P)

        Q_10x10 = np.kron(np.eye(n),Q)

        kf = KalmanFilter(A = F_full, H = H_cf, Q = Q_10x10, R = R_cf, P = P_prev, x0 = x0_cf)

        iterations = 60

        # num_msmts = np.random.randint(0,10,(iterations,5))
        num_msmts = np.array([[0,3,3,3,3] for _ in range(iterations)])

        filter_outputs = run_both_filters(iterations, num_msmts,C,F_full,stations,kf, x)

        errors_df,errors_cf,P_hist_cf,P_hist_df,truth,measurements,predictions_cf,predictions_df = filter_outputs
        squared_error_iteration = lambda i: sum([x**2 for x in errors_df[i,:,0,0]])
        avg += sum([x**2 for x in [squared_error_iteration(i) for i in range(iterations)]])/(n*iterations)
    
    return avg/15

In [227]:
print(MSE([0.225,0.225,0.225,0.225,0.9,0.9,0.9,0.9]))

327.41831574053015


In [270]:
def mse_bfgs(i=0):

  #Linear Constraints:
  A = np.array([[1,0,0,0,0,0,0,0],
              [0,1,0,0,0,0,0,0],
              [0,0,1,0,0,0,0,0],
              [0,0,0,1,0,0,0,0],
              [1,1,1,1,0,0,0,0],
              [0,0,0,0,1,0,0,0],
              [0,0,0,0,1,0,0,0],
              [0,0,0,0,0,1,0,0],
              [0,0,0,0,0,1,0,0],
              [0,0,0,0,0,0,1,0],
              [0,0,0,0,0,0,1,0],
              [0,0,0,0,0,0,0,1],
              [0,0,0,0,0,0,0,1]])
  cons = sp.optimize.LinearConstraint(A, lb = np.zeros(13), ub = np.ones(13))
  bounds = sp.optimize.Bounds(lb = np.zeros(8), ub = np.ones(8))

  #Initial Guess:
  np.random.seed(i+1)
  x0 = [0.15, 0.15, 0.15, 0.15, 1,1,1,1]

  result = sp.optimize.minimize(fun=MSE,x0=x0, bounds=bounds, constraints=cons)

  print("Optimized Weights Vector: ", np.around(result.x, 4))
  print("\nCorresponding Adjacency Matrix:")
  print(convert_to_adj(np.around(result.x, 4)))
  print("\nResulting MSE: ", round(result.fun, 4))

mse_bfgs()

Optimized Weights Vector:  [0.1499 0.1499 0.1504 0.1499 1.     1.     1.     1.    ]

Corresponding Adjacency Matrix:
[[0.3999 0.1499 0.1499 0.1504 0.1499]
 [0.     1.     0.     0.     0.    ]
 [0.     0.     1.     0.     0.    ]
 [0.     0.     0.     1.     0.    ]
 [0.     0.     0.     0.     1.    ]]

Resulting MSE:  39.3966


In [269]:
#1,6,6,6,6
v =  [0.15, 0.15, 0.15, 0.15, 1,1,1,1]
print("Optimized Weight Vector: ", v)
print("Corresponding Adjacency Matrix:\n",convert_to_adj(v))
print("Resulting MSE: ",MSE(v))

Optimized Weight Vector:  [0.15, 0.15, 0.15, 0.15, 1, 1, 1, 1]
Corresponding Adjacency Matrix:
 [[0.4  0.15 0.15 0.15 0.15]
 [0.   1.   0.   0.   0.  ]
 [0.   0.   1.   0.   0.  ]
 [0.   0.   0.   1.   0.  ]
 [0.   0.   0.   0.   1.  ]]
Resulting MSE:  39.38740332943667
